In [ ]:
!pip install facebook_business

In [ ]:
!facebook_business --version

In [ ]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-

import os
import toml
from os.path import basename
import sys
from sys import platform
import json
import time
import requests
from datetime import timedelta, date
from datetime import datetime, timedelta
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign

In [ ]:
with open('fb_config.toml') as conffile:
    config = toml.loads(conffile.read())
    my_app_id = config["my_app_id"]
    my_app_secret = config["my_app_secret"]
    my_access_token = config["my_access_token"]
    id_business = config["id_business"]
    id_sp = config["id_sp"]
    id_pr = config["id_pr"]

FACEBOOK_CLIENT_ID = my_app_id
FACEBOOK_CLIENT_SECRET = my_app_secret
FACEBOOK_ACCESS_TOKEN = my_access_token
FACEBOOK_ACCOUNT_ID = id_sp

In [ ]:
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi

access_token = my_access_token
app_secret = my_app_secret
app_id = my_app_id
id = id_sp
FacebookAdsApi.init(access_token=access_token)

fields = [
  'impressions',
]
params = {
  'breakdown': 'publisher_platform',
}
insights_ = AdSet(id).get_insights(
  fields=fields,
  params=params,
)


In [ ]:
def get_campaign_statistics_for_day(date, account_id):
  date_string = date.strftime("%Y-%m-%d")
  print(date_string)
  ad_account = AdAccount(account_id)
  #print(ad_account)

  fields = [
    'account_name',
    'campaign_name',
    'campaign_id',
    'adset_name',
    'adset_id',
    'ad_name',
    'ad_id',
    'impressions',
    'clicks',
    'spend',
    'actions',
    'reach'
  ]

  params = {
    'breakdowns':['device_platform'],
    'time_range': {
      'since': date_string,
      'until': date_string
    },
    'level': 'ad',
    'limit': 100000
  }

  ad_insights = ad_account.get_insights(fields, params)

  # do whatever you want with the statistics in JSON format
  print(ad_insights) 

  return ad_insights

In [ ]:
if __name__ == '__main__':

  FacebookAdsApi.init(FACEBOOK_CLIENT_ID, FACEBOOK_CLIENT_SECRET, FACEBOOK_ACCESS_TOKEN)
  #my_date = (datetime.now() - timedelta(4))
  my_date = datetime.datetime(2021, 10, 2)
  get_campaign_statistics_for_day(my_date, FACEBOOK_ACCOUNT_ID)

In [ ]:
my_date = datetime.datetime(2021, 10, 2)

In [ ]:
my_app_id = my_app_id
my_app_secret = my_app_secret
my_access_token = my_access_token
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
params = {'date_preset': 'last_7d'}
insights = AdAccount(id_sp).get_insights(params = params)
print(insights)

In [ ]:

from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi

access_token = my_access_token
app_secret = my_app_secret
app_id = my_app_id
id = id_sp
FacebookAdsApi.init(app_id,app_secret,access_token)

fields = [
  'campaign_name'
]
params = {
    'date_preset':'yesterday',
    'level':'ad'
}
get_insights_ = AdSet(id).get_insights(
  fields=fields,
  params=params,
)


In [ ]:
# This program downloads all relevent Facebook traffic info as a csv file
# This program requires info from the Facebook Ads API: https://github.com/facebook/facebook-python-ads-sdk

# Import all the facebook mumbo jumbo
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.business import Business
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adcreative import AdCreative

# Import th csv writer and the date/time function
import datetime
import csv

# Set the info to get connected to the API. Do NOT share this info
my_app_id = my_app_id
my_app_secret = my_app_secret
my_access_token = my_access_token

# Start the connection to the facebook API
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

# Create a business object for the business account
business = Business(id_business)

# Get yesterday's date for the filename, and the csv data
yesterdaybad = datetime.datetime.now() - datetime.timedelta(days=1)
yesterdayslash = yesterdaybad.strftime('%m/%d/%Y')
yesterdayhyphen = yesterdaybad.strftime('%m-%d-%Y')

# Define the destination filename
filename = yesterdayhyphen + '_fb.csv'
filelocation = "/cron/downloads/"+ filename

# Get all ad accounts on the business account
accounts = business.get_owned_ad_accounts(fields=[AdAccount.Field.id])


In [ ]:
accounts = [AdAccount(id_sp),AdAccount(id_pr)]
accounts[0][AdAccount.Field.id]

In [ ]:

# Open or create new file 
try:
    csvfile = open(filelocation , 'w+')
except:
    print ("Cannot open file.")


# To keep track of rows added to file
rows = 0

try:
    # Create file writer
    filewriter = csv.writer(csvfile, delimiter=',')
except Exception as err:
    print(err)

# Iterate through the adaccounts
for account in accounts:
    # Create an addaccount object from the adaccount id to make it possible to get insights
    tempaccount = AdAccount(account[AdAccount.Field.id])

    # Grab insight info for all ads in the adaccount
    ads = tempaccount.get_insights(params={'date_preset':'yesterday',
                                           'level':'ad'
                                          },
                                   fields=[AdsInsights.Field.account_id,
                                           AdsInsights.Field.account_name,
                                           AdsInsights.Field.ad_id,
                                           AdsInsights.Field.ad_name,
                                           AdsInsights.Field.adset_id,
                                           AdsInsights.Field.adset_name,
                                           AdsInsights.Field.campaign_id,
                                           AdsInsights.Field.campaign_name,
                                           AdsInsights.Field.cost_per_outbound_click,
                                           AdsInsights.Field.outbound_clicks,
                                           AdsInsights.Field.spend
                                          ]
    );

    # Iterate through all accounts in the business account
    for ad in ads:
        # Set default values in case the insight info is empty
        date = yesterdayslash
        accountid = ad[AdsInsights.Field.account_id]
        accountname = ""
        adid = ""
        adname = ""
        adsetid = ""
        adsetname = ""
        campaignid = ""
        campaignname = ""
        costperoutboundclick = ""
        outboundclicks = ""
        spend = ""

        # Set values from insight data
        if ('account_id' in ad) :
            accountid = ad[AdsInsights.Field.account_id]
        if ('account_name' in ad) :
            accountname = ad[AdsInsights.Field.account_name]
        if ('ad_id' in ad) :
            adid = ad[AdsInsights.Field.ad_id]
        if ('ad_name' in ad) :
            adname = ad[AdsInsights.Field.ad_name]
        if ('adset_id' in ad) :
            adsetid = ad[AdsInsights.Field.adset_id]
        if ('adset_name' in ad) :
            adsetname = ad[AdsInsights.Field.adset_name]
        if ('campaign_id' in ad) :
            campaignid = ad[AdsInsights.Field.campaign_id]
        if ('campaign_name' in ad) :
            campaignname = ad[AdsInsights.Field.campaign_name]
        if ('cost_per_outbound_click' in ad) : # This is stored strangely, takes a few steps to break through the layers
            costperoutboundclicklist = ad[AdsInsights.Field.cost_per_outbound_click]
            costperoutboundclickdict = costperoutboundclicklist[0]
            costperoutboundclick = costperoutboundclickdict.get('value')
        if ('outbound_clicks' in ad) : # This is stored strangely, takes a few steps to break through the layers
            outboundclickslist = ad[AdsInsights.Field.outbound_clicks]
            outboundclicksdict = outboundclickslist[0]
            outboundclicks = outboundclicksdict.get('value')
        if ('spend' in ad) :
            spend = ad[AdsInsights.Field.spend]

        # Write all ad info to the file, and increment the number of rows that will display
        filewriter.writerow([date, accountid, accountname, adid, adname, adsetid, adsetname, campaignid, campaignname, costperoutboundclick, outboundclicks, spend])
        rows += 1


csvfile.close()

# Print report
print (str(rows) + " rows added to the file " + filename)